In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K

Using TensorFlow backend.


In [2]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
X_train, X_test = X_train[:,:,:,np.newaxis]/255, X_test[:,:,:,np.newaxis]/255

In [3]:
X = X_input = keras.layers.Input(X_train.shape[1:])
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Conv2D(8, (5,5), padding='same', activation='sigmoid')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(16, (5,5), padding='same', activation='sigmoid')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(32, (2,2), padding='valid', activation='sigmoid')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(64, (2,2), padding='valid', activation='sigmoid')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Flatten()(X)
X = keras.layers.Dense(np.max(Y_train)+1, activation='softmax')(X)
M = keras.Model(X_input, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 8)         208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 16)        3216      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 32)          2080      
__________

### search for best cosine annealing max learning rate

In [4]:
hist = M.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=64, epochs=50, callbacks=[
    keras.callbacks.LearningRateScheduler(lambda epoch,lr: 0.1*((epoch+1)/50), verbose=1),
    keras.callbacks.EarlyStopping(monitor='loss', patience=2, verbose=1),
])

Train on 60000 samples, validate on 10000 samples
Epoch 1/50

Epoch 00001: LearningRateScheduler setting learning rate to 0.002.
60000/60000 [==============================] - 12s 195us/step - loss: 0.9203 - acc: 0.7157 - val_loss: 0.2474 - val_acc: 0.9341
Epoch 2/50

Epoch 00002: LearningRateScheduler setting learning rate to 0.004.
60000/60000 [==============================] - 10s 173us/step - loss: 0.1512 - acc: 0.9569 - val_loss: 0.0906 - val_acc: 0.9734
Epoch 3/50

Epoch 00003: LearningRateScheduler setting learning rate to 0.006.
60000/60000 [==============================] - 10s 173us/step - loss: 0.0894 - acc: 0.9736 - val_loss: 0.0634 - val_acc: 0.9801
Epoch 4/50

Epoch 00004: LearningRateScheduler setting learning rate to 0.008.
60000/60000 [==============================] - 10s 173us/step - loss: 0.0680 - acc: 0.9797 - val_loss: 0.0634 - val_acc: 0.9800
Epoch 5/50

Epoch 00005: LearningRateScheduler setting learning rate to 0.010000000000000002.
60000/60000 [===============

### cosine annealing

In [5]:
def cosine_annealing(i, lr_min, lr_max):
    i = i + np.power(2,2)
    log_i = np.log2(i)
    t_min,t_max = np.power(2,np.floor(log_i)), np.power(2,np.floor(log_i)+1)-1
    return lr_min + np.cos(((i-t_min)/(t_max-t_min))*(np.pi/2))*(lr_max-lr_min)
M.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=64, epochs=60, callbacks=[
    keras.callbacks.LearningRateScheduler(lambda epoch,lr: cosine_annealing(epoch,0.012*0.25,0.012), verbose=1)
])

Train on 60000 samples, validate on 10000 samples
Epoch 1/60

Epoch 00001: LearningRateScheduler setting learning rate to 0.012.
60000/60000 [==============================] - 10s 173us/step - loss: 0.0477 - acc: 0.9852 - val_loss: 0.0521 - val_acc: 0.9836
Epoch 2/60

Epoch 00002: LearningRateScheduler setting learning rate to 0.01079422863405995.
60000/60000 [==============================] - 10s 175us/step - loss: 0.0406 - acc: 0.9874 - val_loss: 0.0490 - val_acc: 0.9848
Epoch 3/60

Epoch 00003: LearningRateScheduler setting learning rate to 0.0075000000000000015.
60000/60000 [==============================] - 10s 172us/step - loss: 0.0289 - acc: 0.9909 - val_loss: 0.0443 - val_acc: 0.9868
Epoch 4/60

Epoch 00004: LearningRateScheduler setting learning rate to 0.0030000000000000005.
60000/60000 [==============================] - 11s 177us/step - loss: 0.0159 - acc: 0.9950 - val_loss: 0.0352 - val_acc: 0.9896
Epoch 5/60

Epoch 00005: LearningRateScheduler setting learning rate to 0.01

Epoch 38/60

Epoch 00038: LearningRateScheduler setting learning rate to 0.011080240856136677.
60000/60000 [==============================] - 11s 180us/step - loss: 0.0287 - acc: 0.9905 - val_loss: 0.0538 - val_acc: 0.9841
Epoch 39/60

Epoch 00039: LearningRateScheduler setting learning rate to 0.010869119545301238.
60000/60000 [==============================] - 10s 175us/step - loss: 0.0288 - acc: 0.9908 - val_loss: 0.0548 - val_acc: 0.9863
Epoch 40/60

Epoch 00040: LearningRateScheduler setting learning rate to 0.010637798317452759.
60000/60000 [==============================] - 10s 172us/step - loss: 0.0271 - acc: 0.9912 - val_loss: 0.0488 - val_acc: 0.9855
Epoch 41/60

Epoch 00041: LearningRateScheduler setting learning rate to 0.010386870970865488.
60000/60000 [==============================] - 10s 173us/step - loss: 0.0269 - acc: 0.9913 - val_loss: 0.0478 - val_acc: 0.9859
Epoch 42/60

Epoch 00042: LearningRateScheduler setting learning rate to 0.010116981632439289.
60000/60000 [